In [ ]:
# Libraries for pytho Azure's Cognitive Services interaction
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

# libraries for image preprocessing
from collections import defaultdict
from datetime import datetime
import numpy as np
import cv2
import io

# libraries for python firebase interaction
import pyrebase
import json
from collections import OrderedDict
import collections
from google.cloud import storage
from firebase import firebase
import confg

In [ ]:
camera_location = {"camera 0": (28.4894, 77.0920),"camera 1": (28.5019, 77.0883)}
print(camera_location)

In [ ]:
class surveillance:
    def __init__(self):
        self.wanted_ids = []
        self.face_in_frame = []
        self.wanted_dict = {}
        
        self.KEY = confg.KEY
        self.ENDPOINT = "https://westcentralus.api.cognitive.microsoft.com/"
        self.face_client = FaceClient(self.ENDPOINT, CognitiveServicesCredentials(self.KEY))
        
        self.config = {
          "apiKey": confg.apiKey,
          "authDomain": confg.authDomain,
          "databaseURL": confg.databaseURL,
          "storageBucket": confg.storageBucket,
          "serviceAccount": "admin-sdk.json"
        }
        self.firebase = firebase.FirebaseApplication(confg.databaseURL)
        self.client = storage.Client.from_service_account_json('demo.json')
        self.bucket = self.client.get_bucket(confg.storageBucket)
        self.firebase1 = pyrebase.initialize_app(self.config)
        self.db = self.firebase1.database()
        self.last_update = {}
    
    def stream_handler(self, message):
        var = "status" in message['path']
        action = message['event']
        if action == 'patch':# for on the go data
            Data = message['data']
#             print(Data)
            status = Data['status']
            if status == 0 and var == False:
                while not self.bucket.get_blob(Data['img']):
                    if self.bucket.get_blob(Data['img']):
                        break
                xblob = self.bucket.get_blob(Data['img'])
                url = xblob.public_url
                print("url: ", url)
                detected_faces = self.face_client.face.detect_with_url(url)
                ids = [face.face_id for face in detected_faces]
                i = ids[0]
                self.wanted_ids.append(i)
                self.wanted_dict[Data['id']] = i

            else:
                try:
                    address = message['path']
                    address = address.split('/')
                    pop_img = address[2]
                    list_pop = self.wanted_dict[pop_img]
                    self.wanted_ids.remove(list_pop)
                    del self.wanted_dict[pop_img]
                except:
                    pass
        elif action == 'put' and message['path'] == "/":
            print("Initial data check...")
            try:
                x = message["data"]
                keys = x.keys()
                for key in keys:
                    f_keys = message['data'][key].keys()
                    for f_key in f_keys:
                        inner_dict = message['data'][key][f_key]
                        status = inner_dict['status']
                        if status == 0:
                            blob = self.bucket.get_blob(inner_dict['img'])
                            url = blob.public_url
                            print("url: ", url)
                            detected_faces = self.face_client.face.detect_with_url(url)
                            ids = [face.face_id for face in detected_faces]
                            i = ids[0]
                            self.wanted_ids.append(i)
                            self.wanted_dict[f_key] = i

                        else:
                            #Do Nothing as cause is already solved
                            pass
            except:
                pass
        
    def faces_in_image(self,image, camera_name):
        img = open(image, mode = 'rb')
        detected_faces = self.face_client.face.detect_with_stream(img)
        self.face_in_frame = [face.face_id for face in detected_faces]
        matched_index, confidence = self.check_for_match(camera_name)
        coordinates = [[face.face_rectangle.left, 
                        face.face_rectangle.top,
                        face.face_rectangle.width,
                        face.face_rectangle.height,
                        (0,0,255) if index in matched_index else (0,255,0),
                        confidence
                        ] for index, face in enumerate(detected_faces)]
        return coordinates
    
    def check_for_match(self,camera_name):
        indexes = []
        confidence = []
        for wanted_index, wanted in enumerate(self.wanted_ids):
            for face_index, face in enumerate(self.face_in_frame):
#                 print("wanted: ", wanted, "face: ", face)
                similarity = self.face_client.face.verify_face_to_face(face_id1=wanted, face_id2=face)
#                 print(similarity)
                if similarity.is_identical:
#                     print("Potential match found between: ", wanted_index, 'and', face_index," with confidence: ", similarity.confidence)
                    confidence.append(similarity.confidence)
                    indexes.append(wanted_index)
                    self.match(wanted_index, camera_name )
                else:
                    confidence.append(0)
        return indexes, confidence
                    
    def match(self, index, text):
#         print(text, " detects ", index, " object")
        now = datetime.now()
        now = now.strftime("%m/%d/%Y, %H:%M")
        key = list(self.wanted_dict.keys())[index]
        try:
            lu = self.last_update[key]
            if lu == now:
                pass
                #Do Nothing
            else:
                self.last_update[key] = now
                cam = text
                previous_list1 = self.db.child('policestation').child(key[:3]).child(key).child('seen').get()
                previous_list1 = list(previous_list1.val())
                previous_list1.append(now)
                
                previous_list2 = self.db.child('policestation').child(key[:3]).child(key).child('latitude').get()
                previous_list2 = list(previous_list2.val())
                previous_list2.append(camera_location[text][0])
                
                previous_list3 = self.db.child('policestation').child(key[:3]).child(key).child('longitude').get()
                previous_list3 = list(previous_list3.val())
                previous_list3.append(camera_location[text][1])
                
                self.db.child('policestation').child(key[:3]).child(key).update({"seen":previous_list1})
                self.db.child('policestation').child(key[:3]).child(key).update({"latitude":previous_list2})
                self.db.child('policestation').child(key[:3]).child(key).update({"longitude":previous_list3})
        except KeyError:
            cam = text
            previous_list1 = self.db.child('policestation').child(key[:3]).child(key).child('seen').get()
            previous_list1 = list(previous_list1.val())
            previous_list1.append(now)

            previous_list2 = self.db.child('policestation').child(key[:3]).child(key).child('latitude').get()
            previous_list2 = list(previous_list2.val())
            previous_list2.append(camera_location[text][0])

            previous_list3 = self.db.child('policestation').child(key[:3]).child(key).child('longitude').get()
            previous_list3 = list(previous_list3.val())
            previous_list3.append(camera_location[text][1])
            
            self.db.child('policestation').child(key[:3]).child(key).update({"seen":previous_list1})
            self.db.child('policestation').child(key[:3]).child(key).update({"latitude":previous_list2})
            self.db.child('policestation').child(key[:3]).child(key).update({"longitude":previous_list3})
            self.last_update[key] = now
    
    def update_wanted_ids(self,image):
        img = open(image, 'rb')
        self.wanted_ids.append(self.face_client.face.detect_with_stream(img)[0].face_id)
        print(10*"_-", "wanted updated", 10*"_-")

In [ ]:
ggn = surveillance()

In [ ]:
my_stream = ggn.db.child("policestation").stream(ggn.stream_handler)

flag = True
count = 10
# list of camera addresses
camera_ips = [0,]# "http://192.168.43.178:8080/video"]
# dictionary to store details of camera object and their names
cam_dict = defaultdict(str)

print(10*'_-', "aquiring cameras...", 10*'_-')
for index, ip in enumerate(camera_ips):
    print("aquiring camera "+str(index)+" :", end = '')
    cam = cv2.VideoCapture(ip)
    if (not cam.isOpened()):
        print(' not working!!!')
        cam.release()
        continue
    else:
        print(" ready to go")
    cam_dict[cam] = 'camera ' + str(index)
print(10*'_-', "cameras aquired!!!", 10*'_-')

while flag:
    for camera_id in cam_dict:
        count -= 1
        ok, frame = camera_id.read()
        cv2.imwrite("image.jpg", frame)
        coordinates = ggn.faces_in_image("image.jpg", cam_dict[camera_id])
        if len(coordinates) == 0:
            continue
        for index, [x, y, w, h, color, confidence] in enumerate(coordinates):
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            #cv2.putText(frame, str(confidence[index]), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
        cv2.imshow("live image "+ cam_dict[camera_id], frame)
        k = cv2.waitKey(500)
        if count == 0:
            cv2.waitKey(50000)
        if k & 0xff == ord('q'):
            flag = False
            
cv2.destroyAllWindows()
print("removing cameras...")
for cam in cam_dict:
    print(cam_dict[cam],": ", end = '')
    cam.release()
    print("released")
del cam_dict
#my_stream.close()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
for cam in cam_dict:
    print(cam_dict[cam],": ", end = '')
    cam.release()
    print("released")
del cam_dict        

In [ ]:
my_stream.close()